In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\2\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\2\test'
checkpoint_path = r'C:\Users\24306\Desktop\NASNetLarge\2\NASNetLarge.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[93.0, 13.0, 20.0, 13.0, 37.0, 100.0, 13.0, 52.0, 84.0, 100.0, 84.0, 37.0, 66.0, 23.0, 80.0, 96.0, 10.0, 35.0, 100.0, 84.0, 47.0, 80.0, 16.0, 23.0, 95.0, 100.0, 23.0, 37.0, 32.0, 10.0, 39.0, 96.0, 93.0, 10.0, 95.0, 23.0, 93.0, 13.0, 16.0, 100.0, 47.0, 39.0, 93.0, 13.0, 100.0, 55.0, 80.0, 7.0, 23.0, 95.0, 80.0, 47.0, 40.0, 7.0, 7.0, 37.0, 47.0, 86.0, 32.0, 16.0, 95.0, 47.0, 86.0, 66.0, 32.0, 13.0, 7.0, 55.0, 84.0, 37.0, 10.0, 55.0, 13.0, 37.0, 30.0, 30.0, 37.0, 13.0, 20.0, 37.0, 47.0, 16.0, 16.0, 37.0, 55.0, 80.0, 13.0, 55.0, 37.0, 96.0, 23.0, 95.0, 84.0, 66.0, 47.0, 16.0, 40.0, 86.0, 23.0, 39.0, 66.0, 45.0, 16.0, 37.0, 47.0, 95.0, 37.0, 95.0, 4.0, 45.0, 84.0, 4.0, 84.0, 37.0, 47.0, 32.0, 86.0, 10.0, 66.0, 84.0, 16.0, 95.0, 20.0, 100.0, 4.0, 86.0, 30.0, 47.0, 47.0, 10.0, 37.0, 35.0, 96.0, 80.0, 86.0, 86.0, 45.0, 23.0, 37.0, 80.0, 4.0, 84.0, 13.0, 45.0, 80.0, 7.0, 16.0, 7.0, 4.0, 45.0, 16.0, 30.0, 7.0, 20.0, 66.0, 7.0, 10.0, 39.0, 66.0, 96.0, 30.0, 10.0, 96.0, 84.0, 86.0, 95.0, 20.0, 95.

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
NASNetLarge = tf.keras.applications.nasnet.NASNetLarge(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
NASNetLarge.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = NASNetLarge(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 4032)        84916818  
                                                                 
 global_average_pooling2d (G  (None, 4032)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 4033      
                                                                 
Total params: 84,920,851
Trainable params: 84,724,183
Non-trainable params: 196,668
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('NASNetLarge_loss_2.txt',history.history['loss'])
#要改
np.savetxt('NASNetLarge_valloss_2.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Fri Apr 21 08:50:46 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 170.0141

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\2\NASNetLarge.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\2\NASNetLarge.ckpt\assets


591/591 [==============================] - 386s 583ms/step - loss: 170.0141 - val_loss: 1856.9675
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 3.1812

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\2\NASNetLarge.ckpt\assets


...

591/591 [==============================] - 409s 692ms/step - loss: 0.0016 - val_loss: 0.0218
Epoch 86/300
591/591 [==============================] - 279s 472ms/step - loss: 0.0021 - val_loss: 0.0273
Epoch 87/300
591/591 [==============================] - 277s 468ms/step - loss: 0.0015 - val_loss: 0.0827
Epoch 88/300
591/591 [==============================] - 278s 470ms/step - loss: 0.0014 - val_loss: 0.0809
Epoch 89/300
591/591 [==============================] - 277s 468ms/step - loss: 0.0014 - val_loss: 0.0712
Epoch 90/300
591/591 [==============================] - 277s 468ms/step - loss: 0.0021 - val_loss: 0.0498
Epoch 91/300
591/591 [==============================] - 277s 468ms/step - loss: 0.0012 - val_loss: 0.0262
Epoch 92/300
591/591 [==============================] - 276s 467ms/step - loss: 0.0012 - val_loss: 0.0522
Epoch 93/300
591/591 [==============================] - 277s 469ms/step - loss: 0.0015 - val_loss: 0.0590
Epoch 94/300
591/591 [==============================] - 277

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\2\NASNetLarge.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\2\NASNetLarge.ckpt\assets


591/591 [==============================] - 408s 690ms/step - loss: 0.0119 - val_loss: 0.0135
Epoch 109/300
591/591 [==============================] - 283s 478ms/step - loss: 8.8198e-04 - val_loss: 0.0281
Epoch 110/300
591/591 [==============================] - 281s 475ms/step - loss: 9.6865e-04 - val_loss: 0.0359
Epoch 111/300
591/591 [==============================] - 281s 474ms/step - loss: 0.0011 - val_loss: 0.0407
Epoch 112/300
591/591 [==============================] - 281s 475ms/step - loss: 0.0012 - val_loss: 0.0617
Epoch 113/300
591/591 [==============================] - 280s 474ms/step - loss: 9.7839e-04 - val_loss: 0.0627
Epoch 114/300
591/591 [==============================] - 279s 471ms/step - loss: 8.5064e-04 - val_loss: 0.0920
Epoch 115/300
591/591 [==============================] - 282s 476ms/step - loss: 9.0762e-04 - val_loss: 0.0760
Epoch 116/300
591/591 [==============================] - 280s 474ms/step - loss: 0.0011 - val_loss: 0.1027
Epoch 117/300
591/591 [========

591/591 [==============================] - 281s 476ms/step - loss: 3.3749e-04 - val_loss: 0.0901
Epoch 257/300
591/591 [==============================] - 280s 474ms/step - loss: 2.5775e-04 - val_loss: 0.0768
Epoch 258/300
591/591 [==============================] - 282s 476ms/step - loss: 2.6159e-04 - val_loss: 0.0807
Epoch 259/300
591/591 [==============================] - 282s 476ms/step - loss: 6.0439e-04 - val_loss: 0.0923
Epoch 260/300
591/591 [==============================] - 281s 475ms/step - loss: 2.6999e-04 - val_loss: 0.0710
Epoch 261/300
591/591 [==============================] - 281s 476ms/step - loss: 3.3647e-04 - val_loss: 0.0931
Epoch 262/300
591/591 [==============================] - 280s 474ms/step - loss: 2.3649e-04 - val_loss: 0.0858
Epoch 263/300
591/591 [==============================] - 281s 475ms/step - loss: 2.7597e-04 - val_loss: 0.0730
Epoch 264/300
591/591 [==============================] - 282s 476ms/step - loss: 2.5143e-04 - val_loss: 0.0931
Epoch 265/300
5